# 이미지를 활용해 퀴즈 만들기

### API Key Load

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")  # 환경 변수에서 API 키 가져오기
client = OpenAI(api_key=api_key)  # OpenAI 클라이언트의 인스턴스 생성

### base64 인코딩 함수 정의

In [2]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

### 영어 퀴즈 생성 함수 정의
- 원하는 영어 문제 결과가 포함되지 않을 수 있어서 여러번 시도할 수 있도록 재귀 호출
- 종료 조건 : 최대 시도 회수 도달

In [3]:
def image_quiz(image_path, n_trial=0, max_trial=3):
    if n_trial >= max_trial: # 최대 시도 회수에 도달하면 포기
        raise Exception("Failed to generate a quiz.")

    base64_image = encode_image(image_path) # 이미지를 base64로 인코딩

    quiz_prompt = """
    제공된 이미지를 바탕으로, 다음과 같은 양식으로 퀴즈를 만들어주세요.
    정답은 1~4 중 하나만 해당하도록 출제하세요.
    토익 리스닝 문제 스타일로 문제를 만들어주세요.
    아래는 예시입니다.
    ----- 예시 -----

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
    - (1) 베이커리에서 사람들이 빵을 사고 있는 모습이 담겨 있습니다.
    - (2) 맨 앞에 서 있는 사람은 빨간색 셔츠를 입고 있습니다.
    - (3) 기차를 타기 위해 줄을 서 있는 사람들이 있습니다.
    - (4) 점원은 노란색 티셔츠를 입고 있습니다.

    Listening: Which of the following descriptions of the image is incorrect?
    - (1) It shows people buying bread at a bakery.
    - (2) The person standing at the front is wearing a red shirt.
    - (3) There are people lining up to take a train.
    - (4) The clerk is wearing a yellow T-shirt.

    정답: (4) 점원은 노란색 티셔츠가 아닌 파란색 티셔츠를 입고 있습니다.
    (주의: 정답은 1~4 중 하나만 선택되도록 출제하세요.)
    ======
    """

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": quiz_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # 응답 생성에 사용할 모델 지정
            messages=messages # 대화 기록을 입력으로 전달
        )
    except Exception as e:
        print("failed\n" + e)
        return image_quiz(image_path, n_trial+1)

    content = response.choices[0].message.content

    if "Listening:" in content:
        return content, True
    else:
        return image_quiz(image_path, n_trial+1)

In [6]:
from glob import glob
import json

txt = '' # 문제들을 계속 붙여 나가기 위해 빈 문자열 선언
eng_dict = []
no = 1 # 문제 번호를 위해 선언

for g in glob('./data/quiz/*.jpg'):
    print(g)
    quiz, is_succeed = image_quiz(g)

    if not is_succeed:
        continue


    divider = f'## 문제 {no}\n\n'
    print(divider)

    txt += divider
    # 파일명 추출해 이미지 링크 만들기
    filename = os.path.basename(g) # 마크다운에 표시할 이미지 파일 경로 설정
    txt += f'![image]({filename})\n\n'

    # 문제 추가
    print(quiz)
    txt += quiz + '\n\n---------------------\n\n'

    # 마크다운 파일로 저장
    with open('./data/quiz/image_quiz_eng.md', 'w', encoding='utf-8') as f:
        f.write(txt)

    # 영어 문제만 추출
    eng = quiz.split('Listening: ')[1].split('정답:')[0].strip()

    eng_dict.append({
        'no': no,
        'eng': eng,
        'img': filename
    })

    # json 파일로 저장
    with open('./data/quiz/image_quiz_eng.json', 'w', encoding='utf-8') as f:
        json.dump(eng_dict, f, ensure_ascii=False, indent=4)

    no += 1 # 문제 번호 증가

./data/quiz/bike.jpg
## 문제 1


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?

- (1) 건물 앞에는 많은 오토바이가 주차되어 있습니다.
- (2) 도로에는 차가 여러 대 지나가고 있습니다.
- (3) 건물은 클래식한 외관을 가지고 있습니다.
- (4) 나무들이 건물을 따라 늘어서 있습니다.

Listening: Which of the following descriptions of the image is incorrect?

- (1) There are many motorcycles parked in front of the building.
- (2) Several cars are passing on the road.
- (3) The building has a classical facade.
- (4) Trees are lined up along the building.

정답: (2) 도로에는 차가 여러 대 지나가고 있지 않습니다.
./data/quiz/khf2025.jpg
## 문제 2


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?

- (1) 두 명의 발표자가 무대에 서 있습니다.
- (2) 발표하는 프레젠테이션은 "Healthcare EcoSystem"에 관한 것입니다.
- (3) 청중들이 테이블에 앉아 발표를 듣고 있습니다.
- (4) "2025 메디컬 AI 이노베이션"이라는 문구가 파란색으로 쓰여 있습니다.

Listening: Which of the following descriptions of the image is incorrect?

- (1) Two presenters are standing on the stage.
- (2) The presentation is about the "Healthcare EcoSystem."
- (3) The audience is seated at tables listening to the presentation.
- (4) The phrase "2